# Local Food Wastage Management System

## Project Overview
This project aims to address food wastage by creating a platform that connects surplus food providers (like restaurants and individuals) with receivers (such as NGOs and people in need). This notebook contains all the necessary steps to set up the database, perform data analysis, and launch a fully functional web application using Streamlit.

### Step 1: Setup and Data Loading

First, we import all the necessary Python libraries. We'll use `pandas` for data manipulation, `sqlite3` for the database, and `streamlit` for the web application.

In [1]:
import pandas as pd
import sqlite3
import streamlit as st
import plotly.express as px

Now, we load the four datasets from the provided CSV files into pandas DataFrames.

In [2]:
# Load data from CSV files
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

Let's display the first few rows of the `providers` DataFrame to verify that the data has been loaded correctly.

In [3]:
providers.head()

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


### Step 2: Database Setup and Data Migration

We will use SQLite for our database. It's a lightweight, serverless database that's easy to set up. We'll create a database file named `food_wastage.db` and migrate the data from our DataFrames into corresponding SQL tables. Using `if_exists='replace'` ensures that the notebook can be re-run without causing errors from existing tables.

In [4]:
# Database connection
conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()

# Use pandas to_sql to migrate data to SQLite tables
providers.to_sql('providers', conn, if_exists='replace', index=False)
receivers.to_sql('receivers', conn, if_exists='replace', index=False)
food_listings.to_sql('food_listings', conn, if_exists='replace', index=False)
claims.to_sql('claims', conn, if_exists='replace', index=False)

print("Database and tables created successfully. Data migrated.")

Database and tables created successfully. Data migrated.


### Step 3: SQL Queries and Analysis

Here, we answer the questions outlined in the project requirements document by executing SQL queries against our database. We'll use a helper function to execute queries and return the results as a DataFrame for easy viewing.

In [5]:
def run_query(query):
    return pd.read_sql_query(query, conn)

#### 1. How many food providers and receivers are there in each city?

In [6]:
query1 = """
SELECT City, COUNT(Provider_ID) AS Number_of_Providers
FROM providers
GROUP BY City
ORDER BY Number_of_Providers DESC;
"""
run_query(query1)

,City,Number_of_Providers
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


#### 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

In [7]:
query2 = """
SELECT Provider_Type, SUM(Quantity) AS Total_Quantity_Donated
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity_Donated DESC;
"""
run_query(query2)

,Provider_Type,Total_Quantity_Donated
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116
3,Grocery Store,6059


#### 3. What is the contact information of food providers in a specific city (e.g., 'New Jessica')?

In [8]:
query3 = """
SELECT Name, Address, Contact
FROM providers
WHERE City = 'New Jessica';
"""
run_query(query3)

,Name,Address,Contact
0,Gonzales-Cochran,"74347 Christopher Extensions\nAndreamouth, OK ...",+1-600-220-0480


#### 4. Which receivers have claimed the most food?

In [9]:
query4 = """
SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Name
ORDER BY Total_Claims DESC
LIMIT 10;
"""
run_query(query4)

,Name,Total_Claims
0,William Frederick,5
1,Scott Hunter,5
2,Matthew Webb,5
3,Anthony Garcia,5
4,Kristine Martin,4
5,Kristina Simpson,4
6,Jennifer Nelson,4
7,Donald Caldwell,4
8,Chelsea Powell,4
9,Betty Reid,4


#### 5. What is the total quantity of food available from all providers?

In [10]:
query5 = """
SELECT SUM(Quantity) AS Total_Available_Food_Quantity
FROM food_listings;
"""
run_query(query5)

,Total_Available_Food_Quantity
0,25794


#### 6. Which city has the highest number of food listings?

In [11]:
query6 = """
SELECT Location AS City, COUNT(Food_ID) AS Number_of_Listings
FROM food_listings
GROUP BY Location
ORDER BY Number_of_Listings DESC
LIMIT 1;
"""
run_query(query6)

,City,Number_of_Listings
0,South Kathryn,6


#### 7. What are the most commonly available food types?

In [12]:
query7 = """
SELECT Food_Name, COUNT(*) AS Frequency
FROM food_listings
GROUP BY Food_Name
ORDER BY Frequency DESC
LIMIT 10;
"""
run_query(query7)

,Food_Name,Frequency
0,Rice,114
1,Soup,111
2,Salad,105
3,Dairy,103
4,Chicken,103
5,Pasta,102
6,Bread,98
7,Fish,92
8,Vegetables,91
9,Fruits,81


#### 8. How many food claims have been made for each food item?

In [13]:
query8 = """
SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Number_of_Claims
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Food_Name
ORDER BY Number_of_Claims DESC
LIMIT 10;
"""
run_query(query8)

,Food_Name,Number_of_Claims
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


#### 9. Which provider has had the highest number of successful food claims?

In [14]:
query9 = """
SELECT p.Name AS Provider_Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM providers p
JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
JOIN claims c ON fl.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
LIMIT 1;
"""
run_query(query9)

,Provider_Name,Successful_Claims
0,Barry Group,5


#### 10. What percentage of food claims are completed vs. pending vs. canceled?

In [15]:
query10 = """
SELECT Status, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
FROM claims
GROUP BY Status;
"""
run_query(query10)

,Status,Percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


#### 11. What is the average quantity of food claimed per receiver?

In [16]:
query11 = """
SELECT r.Name, AVG(fl.Quantity) AS Average_Quantity_Claimed
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
JOIN food_listings fl ON c.Food_ID = fl.Food_ID
GROUP BY r.Name
ORDER BY Average_Quantity_Claimed DESC
LIMIT 10;
"""
run_query(query11)

,Name,Average_Quantity_Claimed
0,Thomas Villanueva,50.0
1,Peggy Knight,50.0
2,Nancy Silva,50.0
3,Nancy Jones,50.0
4,Lisa Pitts,50.0
5,Daniel Williams,50.0
6,Christopher Wright,50.0
7,Timothy Patel DVM,49.0
8,Scott Brown,49.0
9,Melissa Little,49.0


#### 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

In [17]:
query12 = """
SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS Number_of_Claims
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Meal_Type
ORDER BY Number_of_Claims DESC;
"""
run_query(query12)

,Meal_Type,Number_of_Claims
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


#### 13. What is the total quantity of food donated by each provider?

In [18]:
query13 = """
SELECT p.Name, SUM(fl.Quantity) AS Total_Donated_Quantity
FROM providers p
JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated_Quantity DESC
LIMIT 10;
"""
run_query(query13)

,Name,Total_Donated_Quantity
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
5,Nelson LLC,142
6,Ruiz-Oneal,140
7,Blankenship-Lewis,124
8,Kelly-Ware,123
9,Bradford-Martinez,121


#### 14. Which food listings are about to expire (e.g., in the next 5 days)?

In [19]:
query14 = """
SELECT Food_Name, Expiry_Date, Quantity
FROM food_listings
WHERE Expiry_Date BETWEEN DATE('now') AND DATE('now', '+5 days')
ORDER BY Expiry_Date ASC;
"""
run_query(query14)

,Food_Name,Expiry_Date,Quantity


#### 15. Which receiver types (NGO, Shelter, etc.) are most active?

In [20]:
query15 = """
SELECT Type, COUNT(c.Claim_ID) AS Number_of_Claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY Type
ORDER BY Number_of_Claims DESC;
"""
run_query(query15)

,Type,Number_of_Claims
0,NGO,272
1,Charity,268
2,Shelter,230
3,Individual,230


### Additional Custom Queries

#### 16. What is the average number of claims per day?

In [21]:
query16 = """
SELECT AVG(daily_claims) AS Average_Claims_Per_Day
FROM (
    SELECT DATE(Timestamp) as claim_date, COUNT(Claim_ID) as daily_claims
    FROM claims
    GROUP BY claim_date
);
"""
run_query(query16)

,Average_Claims_Per_Day
0,1000.0


#### 17. What are the top 5 most common food categories (Vegetarian, Non-Vegetarian, Vegan) listed?

In [22]:
query17 = """
SELECT Food_Type, COUNT(Food_ID) as Number_of_Listings
FROM food_listings
GROUP BY Food_Type
ORDER BY Number_of_Listings DESC;
"""
run_query(query17)

,Food_Type,Number_of_Listings
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


#### 18. Which providers have the highest quantity of a specific food, e.g., 'Bread'?

In [23]:
query18 = """
SELECT p.Name, fl.Quantity
FROM providers p
JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
WHERE fl.Food_Name = 'Bread'
ORDER BY fl.Quantity DESC
LIMIT 5;
"""
run_query(query18)

,Name,Quantity
0,Carr-Wells,50
1,Nelson LLC,50
2,Schmidt Ltd,50
3,"Phelps, Graham and Hayes",50
4,Moore Group,50


#### 19. How many claims are made per receiver type in each city?

In [24]:
query19 = """
SELECT r.City, r.Type, COUNT(c.Claim_ID) AS Number_of_Claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.City, r.Type
ORDER BY r.City, Number_of_Claims DESC;
"""
run_query(query19).head(15)

,City,Type,Number_of_Claims
0,Adamland,Shelter,2
1,Aguilarstad,Individual,2
2,Alexanderbury,Shelter,2
3,Alexatown,Individual,3
4,Allenborough,Shelter,1
5,Allenmouth,Shelter,1
6,Amandaburgh,NGO,1
7,Amandafurt,Individual,1
8,Amandaville,Individual,2
9,Amberfort,Charity,1


#### 20. What is the distribution of provider types (Supermarket, Restaurant, etc.) across the top 5 cities with the most providers?

In [25]:
query20 = """
SELECT City, Type, COUNT(Provider_ID) AS Type_Count
FROM providers
WHERE City IN (
    SELECT City
    FROM providers
    GROUP BY City
    ORDER BY COUNT(Provider_ID) DESC
    LIMIT 5
)
GROUP BY City, Type
ORDER BY City, Type_Count DESC;
"""
run_query(query20)

,City,Type,Type_Count
0,New Carol,Supermarket,2
1,New Carol,Grocery Store,1
2,South Christopherborough,Supermarket,1
3,South Christopherborough,Restaurant,1
4,South Christopherborough,Catering Service,1
5,West Christopher,Grocery Store,2
6,West Lauraborough,Supermarket,1
7,West Lauraborough,Restaurant,1
8,Williamview,Supermarket,1
9,Williamview,Restaurant,1


### Step 4: Streamlit Application Code

The following cell contains the complete code for our Streamlit web application. We use the `%%writefile` magic command to save this code to a file named `app.py`. This application will connect to the `food_wastage.db` database we created earlier.

In [26]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
import plotly.express as px

# Function to create a connection to the SQLite database
def get_connection():
    return sqlite3.connect('food_wastage.db')

# Function to run SQL queries and return a DataFrame
def run_query(query):
    conn = get_connection()
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

st.set_page_config(layout="wide")

st.title("Local Food Wastage Management System")

# Sidebar for navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Home", "Data Viewer", "Analytical Queries", "CRUD Operations"])

# --- Home Page ---
if page == "Home":
    st.header("Welcome to the Food Wastage Management System")
    st.write("""
    This application helps manage surplus food by connecting providers with receivers.
    Use the navigation panel on the left to explore different sections:
    - **Data Viewer**: View the raw data from our database tables.
    - **Analytical Queries**: Explore insights through pre-defined SQL queries and visualizations.
    - **CRUD Operations**: Interact with the database to add, view, update, or delete records.
    """)
    st.image("https://images.unsplash.com/photo-1604162953252-2b604a5a8f4c?q=80&w=2070&auto=format&fit=crop", caption="Connecting Communities, Reducing Waste")

# --- Data Viewer Page ---
elif page == "Data Viewer":
    st.header("Database Table Viewer")
    table_name = st.selectbox("Select a table to view", ["providers", "receivers", "food_listings", "claims"])
    st.dataframe(run_query(f"SELECT * FROM {table_name}"))

# --- Analytical Queries Page ---
elif page == "Analytical Queries":
    st.header("Analytical Queries & Insights")
    
    queries = {
        "1. How many food providers and receivers are there in each city?": "SELECT City, COUNT(Provider_ID) AS Number_of_Providers FROM providers GROUP BY City ORDER BY Number_of_Providers DESC;",
        "2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?": "SELECT Provider_Type, SUM(Quantity) AS Total_Quantity_Donated FROM food_listings GROUP BY Provider_Type ORDER BY Total_Quantity_Donated DESC;",
        "3. What is the contact information of food providers in a specific city (e.g., 'New Jessica')?": "SELECT Name, Address, Contact FROM providers WHERE City = 'New Jessica';",
        "4. Which receivers have claimed the most food?": "SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims FROM receivers r JOIN claims c ON r.Receiver_ID = c.Receiver_ID GROUP BY r.Name ORDER BY Total_Claims DESC LIMIT 10;",
        "5. What is the total quantity of food available from all providers?": "SELECT SUM(Quantity) AS Total_Available_Food_Quantity FROM food_listings;",
        "6. Which city has the highest number of food listings?": "SELECT Location AS City, COUNT(Food_ID) AS Number_of_Listings FROM food_listings GROUP BY Location ORDER BY Number_of_Listings DESC LIMIT 1;",
        "7. What are the most commonly available food types?": "SELECT Food_Name, COUNT(*) AS Frequency FROM food_listings GROUP BY Food_Name ORDER BY Frequency DESC LIMIT 10;",
        "8. How many food claims have been made for each food item?": "SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Number_of_Claims FROM food_listings fl JOIN claims c ON fl.Food_ID = c.Food_ID GROUP BY fl.Food_Name ORDER BY Number_of_Claims DESC LIMIT 10;",
        "9. Which provider has had the highest number of successful food claims?": "SELECT p.Name AS Provider_Name, COUNT(c.Claim_ID) AS Successful_Claims FROM providers p JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID JOIN claims c ON fl.Food_ID = c.Food_ID WHERE c.Status = 'Completed' GROUP BY p.Name ORDER BY Successful_Claims DESC LIMIT 1;",
        "10. What percentage of food claims are completed vs. pending vs. canceled?": "SELECT Status, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage FROM claims GROUP BY Status;",
        "11. What is the average quantity of food claimed per receiver?": "SELECT r.Name, AVG(fl.Quantity) AS Average_Quantity_Claimed FROM receivers r JOIN claims c ON r.Receiver_ID = c.Receiver_ID JOIN food_listings fl ON c.Food_ID = fl.Food_ID GROUP BY r.Name ORDER BY Average_Quantity_Claimed DESC LIMIT 10;",
        "12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?": "SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS Number_of_Claims FROM food_listings fl JOIN claims c ON fl.Food_ID = c.Food_ID GROUP BY fl.Meal_Type ORDER BY Number_of_Claims DESC;",
        "13. What is the total quantity of food donated by each provider?": "SELECT p.Name, SUM(fl.Quantity) AS Total_Donated_Quantity FROM providers p JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID GROUP BY p.Name ORDER BY Total_Donated_Quantity DESC LIMIT 10;",
        "14. Which food listings are about to expire (e.g., in the next 5 days)?": "SELECT Food_Name, Expiry_Date, Quantity FROM food_listings WHERE Expiry_Date BETWEEN DATE('now') AND DATE('now', '+5 days') ORDER BY Expiry_Date ASC;",
        "15. Which receiver types (NGO, Shelter, etc.) are most active?": "SELECT Type, COUNT(c.Claim_ID) AS Number_of_Claims FROM receivers r JOIN claims c ON r.Receiver_ID = c.Receiver_ID GROUP BY Type ORDER BY Number_of_Claims DESC;",
        "16. What is the average number of claims per day?": "SELECT AVG(daily_claims) AS Average_Claims_Per_Day FROM (SELECT DATE(Timestamp) as claim_date, COUNT(Claim_ID) as daily_claims FROM claims GROUP BY claim_date);",
        "17. What are the top 5 most common food categories (Vegetarian, Non-Vegetarian, Vegan) listed?": "SELECT Food_Type, COUNT(Food_ID) as Number_of_Listings FROM food_listings GROUP BY Food_Type ORDER BY Number_of_Listings DESC;",
        "18. Which providers have the highest quantity of a specific food, e.g., 'Bread'?": "SELECT p.Name, fl.Quantity FROM providers p JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID WHERE fl.Food_Name = 'Bread' ORDER BY fl.Quantity DESC LIMIT 5;",
        "19. How many claims are made per receiver type in each city?": "SELECT r.City, r.Type, COUNT(c.Claim_ID) AS Number_of_Claims FROM receivers r JOIN claims c ON r.Receiver_ID = c.Receiver_ID GROUP BY r.City, r.Type ORDER BY r.City, Number_of_Claims DESC;",
        "20. What is the distribution of provider types (Supermarket, Restaurant, etc.) across the top 5 cities with the most providers?": "SELECT City, Type, COUNT(Provider_ID) AS Type_Count FROM providers WHERE City IN (SELECT City FROM providers GROUP BY City ORDER BY COUNT(Provider_ID) DESC LIMIT 5) GROUP BY City, Type ORDER BY City, Type_Count DESC;"
    }

    query_choice = st.selectbox("Select a Query", list(queries.keys()))
    
    if query_choice:
        result_df = run_query(queries[query_choice])
        st.write("**Query Result:**")
        st.dataframe(result_df)

        st.write("**Visualization:**")
        try:
            if "Percentage" in result_df.columns:
                fig = px.pie(result_df, names=result_df.columns[0], values='Percentage', title=query_choice)
            elif "Total_Quantity_Donated" in result_df.columns or "Total_Donated_Quantity" in result_df.columns or "Number_of_Claims" in result_df.columns or "Frequency" in result_df.columns or "Number_of_Listings" in result_df.columns or "Number_of_Providers" in result_df.columns:
                fig = px.bar(result_df, x=result_df.columns[0], y=result_df.columns[1], title=query_choice)
            else:
                st.warning("No suitable visualization for this query.")
                fig = None
            if fig:
                st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Could not generate visualization: {e}")


# --- CRUD Operations Page ---
elif page == "CRUD Operations":
    st.header("Database Management (CRUD)")
    
    table_to_manage = st.selectbox("Select Table to Manage", ["providers", "receivers"])
    operation = st.radio("Select Operation", ["Create", "Read/Filter", "Update", "Delete"])

    conn = get_connection()
    cursor = conn.cursor()

    if operation == "Create":
        st.subheader(f"Add a New Record to `{table_to_manage}`")
        if table_to_manage == 'providers':
            with st.form(key='add_provider_form'):
                name = st.text_input("Name")
                ptype = st.text_input("Type")
                address = st.text_area("Address")
                city = st.text_input("City")
                contact = st.text_input("Contact")
                submit_button = st.form_submit_button(label='Add Provider')

                if submit_button:
                    cursor.execute('INSERT INTO providers (Name, Type, Address, City, Contact) VALUES (?,?,?,?,?)', (name, ptype, address, city, contact))
                    conn.commit()
                    st.success("Provider added successfully!")
        
    elif operation == "Read/Filter":
        st.subheader(f"View and Filter Data from `{table_to_manage}`")
        filter_col = st.selectbox("Filter by Column", pd.read_sql(f'PRAGMA table_info({table_to_manage});', conn)['name'].tolist())
        filter_val = st.text_input(f"Value for {filter_col}")
        if st.button("Filter"):
            st.dataframe(run_query(f"SELECT * FROM {table_to_manage} WHERE {filter_col} LIKE '%{filter_val}%'"))

    elif operation == "Update":
        st.subheader(f"Update a Record in `{table_to_manage}`")
        id_col = 'Provider_ID' if table_to_manage == 'providers' else 'Receiver_ID'
        record_id = st.number_input(f"Enter {id_col} of the record to update", min_value=1, step=1)
        if table_to_manage == 'providers':
            new_contact = st.text_input("Enter new contact information")
            if st.button("Update Contact"):
                cursor.execute(f'UPDATE providers SET Contact = ? WHERE Provider_ID = ?', (new_contact, record_id))
                conn.commit()
                st.success(f"Provider {record_id} updated successfully!")

    elif operation == "Delete":
        st.subheader(f"Delete a Record from `{table_to_manage}`")
        id_col = 'Provider_ID' if table_to_manage == 'providers' else 'Receiver_ID'
        record_id_del = st.number_input(f"Enter {id_col} of the record to delete", min_value=1, step=1)
        if st.button("Delete Record"):
            cursor.execute(f'DELETE FROM {table_to_manage} WHERE {id_col} = ?', (record_id_del,))
            conn.commit()
            st.warning(f"Record with ID {record_id_del} deleted from `{table_to_manage}`!")

    conn.close()


Overwriting app.py


### Step 5: Running the Streamlit Application

Now that the `app.py` file has been created, you can run the cell below to launch the web application. A new browser tab should open with the running application. If it doesn't, copy the 'Network URL' from the output and paste it into your browser.

In [ ]:
!python -m streamlit run app.py